# Unit 4: The `pandas` DataFrame
--------------------------------

Unit 2 introduced the `DataFrame` object for loading data and computing descriptive statistics. We'll take this a step further and demonstrate the use of pivot tables and other techniques. If you've used pivot tables in a spreadsheet program, the concept is similar: in a pivot table, data is summarized based on one or more categorical variables. This can be a quick way to compute averages or other descriptive statistics for each category in a dataset. When the pivot table (or other data processing) is performed in Python, the results can be easily plotted in `matplotlib` or saved to a file.

**After completing this unit, you should be able to:**

- Select and filter data in a `pandas.DataFrame`
- Create new columns by applying functions to existing data
- Create and save a pivot table

## 4.1. Selecting and filtering data

In Unit 2 we introduced the `pandas.DataFrame` object, including how to load data and basic selections by column or index. We'll start by reviewing the layout of the `DataFrame` in more depth. The data frame can be visualized as shown in the image below, where the data is organized in terms of an *index* (the rows) and *columns*. By default, the index will just be a row number, but it can also be customized as we'll see later with pivot tables.

![DataFrame layout](../../assets/DataFrame.svg)

The following example was introduced in Unit 2.

In [ ]:
import pandas as pd
import numpy as np

# read a .csv file into a DataFrame variable named df
fid_df = pd.read_csv('../../data/pcr-polyethylene_gc-fid.csv')

# display the top n rows in the DataFrame named df (defaults to n=5 if blank)
fid_df.head(5)

Access the index with the `DataFrame.index` property. In this example the default index is a range of integers. Similar to the `numpy.arange()` that we used to create an array in Unit 2, this range is defined with start, stop and step values.

In [ ]:
fid_df.index

There are 3 ways that we can select rows from the `DataFrame` object: 

1. Select by the data *value*
2. Select by the index *number* (`iloc`)
3. Select by the index *value* (`loc`)

To begin, let's select rows based on the value of the time. This data represents the flame ionization detector (FID) signal from a gas chromatograph. This is a time series (in decimal minutes) and the FID sensor data. We might know that the true data begins at 6 minutes, so we use the code `fid_df[fid_df['minutes'] > 6]` to filter the results to times that are greater than 6 minutes.

The notatation for this filter may look strange, so let's work our way from the inside out. 

The first piece of this code 

```
fid_df['minutes']
```

selects the minutes column, which contains the data that we want to use for our filter. We learned about this in Unit 2. Now, we take the data in that column and run the test to see if it is greater than our chosen start time, 6 minutes.

```
fid_df['minutes'] > 6
```

This comparison returns a series of the same shape as `fid_df['minutes']`, with a value of `True` if this time is greater than 6 or `False` if the time is less than or equal to 6. Note that we could have used any other `True`/`False` test, including `<`, `<=`, `>=`, `==` (equal), `!=` (not equal). Finally, this series of `True`/`False` values is passed to the `DataFrame`.

```
fid_df[fid_df['minutes'] > 6]
```

Any rows where the series returned by the test `fid_df['minutes'] > 6` has returned `True` are part of the selected rows.

In [ ]:
fid_df[fid_df['minutes'] > 6]

After the filter is applied you should find that there are now 46983 rows, compared to 53184 in the original `DataFrame`. These rows have not been deleted -- they still exist in the `fid_df` object. What is returned is a slice of the orginal `DataFrame`.

Now, we might want to apply multiple filters using boolean logic to return values where the time is both greater than 6 *and* less than 7 minutes. The first thing to note in the example below is that each condition is enclosed within parentheses. This is required, because the order of operations is not well-defined for this complicated set of instructions. Second, we use the bitwise *and* operator `&` to indicate that both conditions must be `True` for the result to be `True`. There are additional operators for bitwise *or* `|` and bitwise exclusive or (*xor*) `^`. The *xor* operation means that only one of the conditions may be `True` (one, but not both).

Truth table:

| Value 1 | Operator | Value 2 | Result |
|---------|----------|---------|--------|
| `True` | `&` | `True` | `True` |
| `True` | `&` | `False` | `False` |
| `False` | `&` | `True` | `False` |
| `False` | `&` | `False` | `False` |
| `True` | `\|` | `True` | `True` |
| `True` | `\|` | `False` | `True` |
| `False` | `\|` | `True` | `True` |
| `False` | `\|` | `False` | `False` |
| `True` | `^` | `True` | `False` |
| `True` | `^` | `False` | `True` |
| `False` | `^` | `True` | `True` |
| `False` | `^` | `False` | `False` |

Experiment with the code below to apply different filters to this data.

In [ ]:
fid_df[(fid_df['minutes'] > 6) & (fid_df['minutes'] < 7)]

The next two ways of selecting rows deal with the row index number and row index value. In our current example, these are interchangeable because the index is a series of integers that are identical to the row number. So, the `loc` and `iloc` properties will work the same. To make it more clear what the difference is between these two, reload the data into a new `DataFrame`. This time, set the row index to be the *minutes* column from our data set.

In [ ]:
# load the data using the minutes as the row index
fid_df_timeindexed = pd.read_csv('../../data/pcr-polyethylene_gc-fid.csv', index_col='minutes')
fid_df_timeindexed.head(5)

Observe that the index is no longer a range but an array of floating-point (real) numbers.

In [ ]:
fid_df_timeindexed.index

To select rows based on row number (even though the index values are now the time in minutes), use the `DataFrame.iloc` property and our array slicing notation. This example selects the first 5 rows from the `DataFrame`, just like using the `DataFrame.head(5)` function.

In [ ]:
fid_df_timeindexed.iloc[:5]

Another way to select individual rows is to provide a list (or array) of row index numbers to `iloc`. Note the double square brackets - the inner brackets indicate that we are creating a list with the values `1`, `3` and `5`. This list is passed to the `iloc` accessor to select the individual rows.

In [ ]:
fid_df_timeindexed.iloc[[1, 5, 10]]

Similar the previous lessons on list or array slicing, even-numbered rows can be selected using the following sequence: 

In [ ]:
fid_df_timeindexed.iloc[::2]

Regardless of what the index looks like, rows can always be selected using the `iloc` accessor. In contrast, the `loc` accessor slices based on the *value* in the index column, which may not be an integer value. To see how this works, revisit our previous example where the range was filtered to time values beginning at 6 minutes. Because the index is now defined as the time values, the simple slicing notation can be used to select all points where the time is greater than 6 minutes.

In [ ]:
fid_df_timeindexed.loc[6:]

Or, the time interval between 6 and 7 minutes by adding an ending value in the slicing notation.

In [ ]:
fid_df_timeindexed.loc[6:7]

This slicing notation is very compact to write and can be convenient. However, it does lack some of the flexibility of the column-based filters that we introduced earlier.

## 4.2. Applying a function to create a new column

Creating a new column in a `DataFrame` can be be easily done. Perhaps the time from our GC dataset should be displayed in units of seconds, instead of minutes, as demonstrated in Unit 2. We can simply select the `minutes` column as before, multiply the entire column by 60, and assign the result to a new column called `seconds`. The `pandas` code will take care of creating this new column for us, even though it didn't exist previously. 

In [ ]:
fid_df['seconds'] = fid_df['minutes'] * 60
fid_df.head(5)

Additionally, we can easily apply operations such as differencing or moving-window functions. *Differencing* is a common technique in time series analysis to de-trend data. In this operation, for each row in the data series, the previous value is subtracted so that only the difference between the two sequencial points remains. The result is a new data series, where $\text{diff}(x)_i=x_i-x_{i-1}$. Because this function looks backward in the dataset by one index, there is no value for the first element in the series. It does not wrap around to to the other end of the array. In this example, observe that the row 0 is populated with `NaN`, meaning *not a number*.

In [ ]:
fid_df['fid'].diff()

The `diff()` function can also be used to calculate a numerical derivative from a data set. In the example below.

$$\frac{\Delta signal}{\Delta time}=\frac{\texttt{fid\_df[`fid'].diff()}}{\texttt{fid\_df[`seconds'].diff()}}$$

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# calculate the first derivative (change in FID signal) / (change in time)
fid_df['1st_derivative'] = fid_df['fid'].diff() / fid_df['seconds'].diff()

# plot the FID signal and its 1st derivative
# the \n in the label is the new-line character to split the label into 2 lines
fig, ax = plt.subplots(nrows=2, sharex=True)

ax[0].plot(fid_df['seconds'], fid_df['fid'])
ax[0].set_ylabel('Original\nFID Signal')

ax[1].plot(fid_df['seconds'], fid_df['1st_derivative'])
ax[1].set_xlabel('Seconds')
ax[1].set_ylabel('1st Derivative\nof FID Signal')

Another important tool in analyzing time-series data is the moving-average operation. This can reduce noise in the data, smoothing the data series. Sometimes, a smoothing operation (either a moving average, or a more sophisticated signal filter) will be applied to the data prior to computing a derivative. This can make the resulting derivative much less noisy.

The `Series.rolling(n)` function creates a moving window of $n$ rows. We apply a function, such as `Series.mean()` to this window to average the datapoints within the window. By default, this is a trailing window. In the example below, notice that the first available value is in row index 4 (the 5th row), and this contains the `Series.mean()` of this row together with the previous 4 rows.

In [ ]:
fid_df['fid'].rolling(5).mean().head(10)

As the size of the window increases, the trailing moving average may appear to be out of sync with the data. The data can be shifted to center the average in the middle of the window with the `center=True` parameter of the `Series.rolling()` function. Now, the `NaN` values are split between the start and end of the series.

In [ ]:
fid_df['fid'].rolling(5, center=True).mean().head(10)

In [ ]:
fid_df['fid'].rolling(5, center=True).mean().tail(10)

The moving average can be used to create a new column in our `DataFrame` with the signal smoothed out. This operation removes the high peaks from the data, which may or may not be appropriate for your analysis. 

In [ ]:
fid_df['rolling_mean'] = fid_df['fid'].rolling(51, center=True).mean()

fig, ax = plt.subplots()
ax.plot(fid_df['seconds'], fid_df['fid'], label='Original FID Signal')
ax.plot(fid_df['seconds'], fid_df['rolling_mean'], label='Rolling Mean')

ax.legend()


Finally, save the results of this analysis to a comma-separated (csv) text file. Adding the `index=False` parameter will exclude the integer row index from the output file. 

In [ ]:
fid_df.to_csv('../../output/unit4_fid.csv', index=False)

## 4.3. Creating and saving a pivot table

### 4.3.1. Long versus wide data sets

Pivot tables are most useful on long-form datasets, which have a minimum number of columns. Consider the dataset below, which contains measurements of different properties of polymer films. In a *wide* form, there might be one column per property, and then one row per film. In *long* form there is a single column containing all of the measured values. To clarify what the measured value represents, there is a categorical value (as a new column) to define specifically what property was measured. This data structure can be more flexible to adding new properties in the future, especially if the data is being stored in a relational database. Rather than adding new columns each time a test is added, we can just add new rows with the new `Property` value. This form also allows for different numbers of replicate measurements for each film property, while not introducing any blank spaces in the data.

In [ ]:
film_df = pd.read_excel('../../data/film_testing.xlsx', sheet_name='physical_properties')
film_df.head(20)

### 4.3.2. Creating the pivot table

A pivot table can summarize these results, averaging over the individual polymer grades within each polymer family. Look at the parameters in the [`DataFrame.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) function below. The column named `Measurement` is used to calculate the values in the pivot table - this is where the numerical data is stored. The column named `FilmType` is used to generate the row index. There will be a row for each unique value from this column. The column named `Property` (what we are measuring) will become a column heading in the pivot table, with one column for each unique value of a property. Finally, the `aggfunc` parameter indicates what mathematical operation we wish to apply to the data values. Optional values are: `mean`, `max`, `min`, `count`, `stdev`. 

The output of the `DataFrame.pivot_table()` function is another `DataFrame`, with the row indices and column names as defined.

In [ ]:
film_pt = film_df.pivot_table(values='Measurement', index='FilmType', columns='Property', aggfunc='mean')
film_pt

Because this is a `DataFrame`, the same rules for indexing and column selection apply. 

In [ ]:
film_pt['Tensile Modulus']

In [ ]:
film_pt['Tensile Modulus'].loc['BOPP']

Pivot tables can get more detailed (and complicated) by using lists of multiple columns for the `index` or `columns` parameters. Look at the example below and see how these values are nested. The rows are first clustered by the `FilmType` value, and then by `FilmID`. Also have two levels for the test method, and the direction in which the film is tested (**m**achine **d**irection / **t**ransverse **d**irection).

In [ ]:
film_pt2 = film_df.pivot_table(values='Measurement', index=['FilmType', 'FilmID'], columns=['Property', 'Direction'], aggfunc='mean')
film_pt2

Selecting rows and columns which are nested can be easily done, by separating the individual selections by comma.

In [ ]:
film_pt2['Tensile Modulus', 'MD']

One additional step is to plot the results of a pivot table. In the example below, the average modulus for high-density polyethylene (HDPE) and polypropylene (PP) polymers is plotted. The values from the pivot table index are used to label the bars in this plot.

In [ ]:
fig, ax = plt.subplots()

# create a range of values for x, since our index is not numeric
x = np.arange(len(film_pt))

# plot the values, applying the index values as labels
ax.bar(x, film_pt['Tensile Modulus'], tick_label=film_pt.index)

# label the numerical axis
ax.set_ylabel('Tensile Modulus (MPa)')

### 4.3.3. Saving the results

Again, the pivot table is just another `DataFrame`, so we can save the results. In the empty cell below, practice saving the `film_pt2` pivot table to a file in the `output` directory. Open the file to verify that it saved correctly.

--------------
## Next Steps:

1. Complete the [Unit 4 Problems](./unit04-problems.ipynb) to test your understanding
2. Advance to [Unit 5](../05-statistics/unit05-lesson.ipynb) when you're ready for the next step